In [6]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
#from googletrans import Translator
from translate import Translator
from deep_translator import GoogleTranslator

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)

In [7]:
train = pd.read_csv('data/train_translated.csv')

In [4]:
# suffisticated approach for translation
def translate(data, column):
    original = data[column].dropna().unique()
    translated = [GoogleTranslator(source='et', target='en').translate(word) for word in original]
    print('done')
    data[column] = data[column].replace(original, translated)

In [5]:
translate(train, 'state')

done


In [27]:
train = train.drop(['Unnamed: 0'], axis=1)

In [28]:
train.set_index('id', inplace=True)

In [30]:
train.to_csv('data/train_translated.csv')

In [8]:
train.tail(200)

,id,full_nr,name,ks,material,commentary,event_type,location,start,end,before_Christ,country_and_unit,participants_role,participant,parish,text,class,technique,parameter,unit,value,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,collection_queue_nr,collection_additional_nr,element_count,legend,is_original,initial_info,damages,state,color,additional_text,type
13800,2080468,AM N 17466,Female workers from a candy factory,17466.0,photographic material>film>acetate cellulose film,NaN,preparation,NaN,1940,NaN,NaN,NaN,the author,"Lukin, E.",NaN,NaN,NaN,photography>black and white photography,negative format,24 x 36 mm,1,AM,NaN,NaN,NaN,NaN,N,NaN,NaN,1.0,NaN,1.0,NaN,NaN,good,NaN,NaN,photo negative
13801,2793399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,N,6.0,NaN,1.0,NaN,1.0,NaN,NaN,good,NaN,NaN,photo negative
13802,4120141,AM _ 41223:15 M,dirham,NaN,silver,NaN,preparation,the city of Andaraba,920,921,ei,Arabic,NaN,NaN,NaN,NaN,NaN,NaN,diameter,mm,29,AM,_,41223.0,15.0,NaN,M,NaN,NaN,1.0,"Mündiaare, found in Tartu County, Peipsiääre municipality, Pusi village in 2010. MKA Archaeological findings\ntransfer-acceptance act No. 5.1-15/59",1.0,The Samanids,NaN,good,NaN,text,coin
13803,2982090,ETMM _ 9554 Mo 10:1/32:33,"Program, Symphony concert, dir.P.\nLilje, ETV 25 anniversary concert",10.0,paper,NaN,culture,NaN,27.09.1980,NaN,ei,NaN,conductor,"Lilje, Peeter",NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETMM,_,9554.0,NaN,NaN,Mo,1.0,32:33:00,1.0,NaN,1.0,NaN,NaN,good,NaN,NaN,plan
13804,2865399,NaN,NaN,NaN,NaN,NaN,theater,NaN,1960,NaN,ei,NaN,imagined,"Järvet, Jüri",NaN,NaN,NaN,black and white photography,NaN,NaN,NaN,ETMM,_,12709.0,885.0,NaN,T,3.0,06:29,1.0,Mikk Mikiver's photo collection handed over by Carmen Mikiver ca 2010,1.0,NaN,NaN,good,NaN,NaN,photo
13805,197740,ETMM _ 10331 M 21:2/114:2,"""Night piece"" for string quartet",21.0,paper,NaN,music,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,handwriting,NaN,NaN,NaN,ETMM,_,10331.0,NaN,NaN,M,2.0,114:02:00,1.0,NaN,1.0,NaN,NaN,good,multicolored,Ööpala see E.Aava fund no.2,manuscript
13806,2538832,AM Fm 316:290,"Shooting of the movie ""Dangerous Games"".",316.0,photographic material>film,NaN,taking pictures,NaN,25.05.1973,18.04.1974,ei,Estonia,the author,"Reimann, Villu",NaN,NaN,NaN,photography>black and white photography,negative format,60 x 60 mm,1,AM,NaN,NaN,NaN,NaN,Fm,290.0,NaN,1.0,"Addendum to Act No. 2013:7395 (15.08.2013). Partially preserved collection of negatives from Tallinnfilm's photo archive - coverage of two films: ""Indrek"", ""Dangerous Games"".",NaN,NaN,NaN,good,NaN,NaN,photo negative
13807,2728231,NaN,NaN,NaN,NaN,NaN,collection activities of the museum,NaN,20.09.1983,30.09.1983,ei,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,F,1.0,NaN,1.0,NaN,1.0,NaN,NaN,good,NaN,NaN,photo
13808,3346501,AM _ 6740 G 1977,"graphics, General J.A. Golovin",1977.0,paper,NaN,portrait of a man,NaN,NaN,NaN,NaN,NaN,the author,"Krüger, Karl Ferdinand Wilhelm",NaN,NaN,NaN,lithography,height,cm,44.5,AM,_,6740.0,NaN,NaN,G,NaN,NaN,1.0,NaN,1.0,NaN,"yellowed, dirty, damp-stained",satisfactory,NaN,NaN,graphics
13809,2406273,AM _ 2549:3-R F 1688,Exhibition organized on the occasion of the 4th ECB Congress,1688.0,silver gelatin paper,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,black and white photography,NaN,NaN,NaN,AM,_,2549.0,3.0,R,F,NaN,NaN,1.0,NaN,1.0,NaN,NaN,good,NaN,NaN,photo
